## PRÁCTICA 1.B

JAVIER GÓMEZ MORALEDA

MARIO ALCOLEA ALCOLEA

GRUPO 17

Los problemas que hemos resuelto han sido el 1 y el 2. Al principio de cada uno hay una explicación sobre como lo hemos estructurado y como funcionan sus métodos. Además, hemos realizado comentarios en algunas partes del código para hacerlo más claro.

# CLASE PROBLEMA

In [16]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

In [17]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search, uniform_cost_search, best_first_graph_search

# PROBLEMA 1: LA LINTERNA Y EL PUENTE

__Introducción y entrada del problema:__

Este problema se encarga de buscar una solución para que todos crucen un puente de izquierda a derecha con la limitación de que a lo sumo pueden pasar 2 a la vez y que el tiempo de la linterna que necesitan para cruzar es limitado. En primer lugar, la entrada inicial a este problema consiste en una tupla de n elementos en la que están representados el tiempo que tarda en pasar cada persona salvo el último elemento que es el tiempo de la linterna. En nuestro caso, la inicialización será, como se ve mas abajo, una tupla de esta forma: Linterna((10, 30, 60, 80, 120, 300)). Sin embargo, podemos poner el tiempo que queramos a cada persona y aumentar el número de personas que quieren cruzar (teniendo en cuenta que esto aumentará el tiempo de ejecución) o el tiempo de la linterna, pero lo que no podremos hacer es tener varios puentes y/o linternas.

__Representación de los estados:__

Cada estado constará de una tupla en la que representaremos con 0 o 1 si la persona con ese índice se encuentra a la izquierda o derecha del puente. Hemos asignado 1 a la izquierda (objetivo) y 0 a la derecha. Además, el penúltimo elemento de la tupla será donde se encuentra la linterna y el último su tiempo restante. Para saber cuanto tiempo tarda cada persona en cruzar, tendremos un array tiempo_personas que guardará el tiempo de cada persona y puesto que nunca es modificado, no se incluye en los estados. En nuestro ejemplo, el estado inicial es (0, 0, 0, 0, 0, 0, 300) y el array tiempo_personas = [10, 30, 60, 80, 120].

__Función init:__

Esta función incializa los estados y las variables globales en funcion de una tupla entrada_inicial. De ella extraigo el tiempo de personas que guardo en el array tiempo_personas y creo una tupla del estado inicial con todo ceros (al principio todos están a la derecha). A ese estado le sumo el tiempo de la linterna.

__Función actions:__

En esta función distingo dos casos particulares. En función de la ubicación de la linterna, tengo unas opciones u otras. Si la linterna está en la parte derecha, compruebo todas las personas que también lo están (por eso no llego las dos ultimas posiciones del estado), y con un doble bucle for, voy creando todas las posibilidades de emparejamientos posibles devueltos en pares de indices (i,j). Esto quiere decir que la persona con índice i y la persona con índice j están a la izquierda y podrían cruzar juntos. Si la linterna está en la parte izquierda es aún más sencillo ya que devuelvo un par (i, i) (para simplificar) en el cual quiere decir que la persona i está en la parte izquierda y puede cruzar a la derecha. Es importante señalar que SÓLO vuelve una persona ya que como solo cruzan de 2 en 2, no tendría sentido que cruzasen 2 a la izquierda y ambos volviesen a la derecha. Por último, antes de añadir la acción a las acciones posibles, compruebo si ese estado me va a llevar a un tiempo de la linterna negativo. En el caso en el que así sea, no lo añado a los estados posibles. Es por esto por lo que la función goal_test se simplifica mucho.

__Función result:__ 

Es el resultado de aplicar una acción (i,j) o (i,i) a un estado. Como siempre es un par, extraigo dos variables con los índices de las dos (o una) personas. Otra vez distingo el caso en el que la linterna esté a la derecha o a la izquierda. Si están a la derecha cambio el estado poniendo sus posiciones a 1. Si están a la izquierda y como sólo vuelve una persona, cambio su estado a 0. En ambos casos se actualiza la posición donde se encuentra la linterna.

__Función goal_state:__ 

Esta función únicamente comprueba que todas las posiciones del estado (salvo la última que es el tiempo), se encuentren a 1. Eso quiere decir que hemos dado con la solución y todos han conseguido cruzar. Se queda así de simple ya que no nos hace falta comprobar si el tiempo es mayor o igual que 0, ya que como he dicho antes, en las acciones posibles sólo se generan opciones válidas.


In [18]:
class Linterna(Problem):
           
    tiempo_personas = []
    
    #Variables aclaratorias
    izquierda = 1
    derecha = 0
    
    def __init__(self, entrada_inicial):
        
        #Extraigo tiempo de la linterna
        tiempo_linterna = entrada_inicial[len(entrada_inicial) - 1]
        
        #Tiempo de las personas
        self.tiempo_personas = []
        
        #Array estado inicial
        initial_array = []
        
        #Guardo el tiempo de las personas
        for i in range(len(entrada_inicial) - 1):
            self.tiempo_personas.append(entrada_inicial[i])
                
        #Inicializo el array del estado inicial a 0
        for i in range(len(entrada_inicial)):
            initial_array.append(0)
            
        #Incluyo el tiempo de la linterna
        initial_array.append(tiempo_linterna)
        
        initial = tuple(initial_array)
        
        self.initial = initial
       
    def actions(self, state):
        
        accs=list()
        
        #Extraigo la ubicación de la linterna accediendo a la penúltima posición del estado
        ubicacion_linterna = state[len(state) - 2]

        #Si la linterna se encuentra a la derecha del puente
        if(ubicacion_linterna == self.derecha):
            #Primer buclce que recorro las personas
            for i in range(len(state) - 2):
                if(state[i] == self.derecha):
                    #Segundo bucle para buscar parejas
                    for j in range(i+1, len(state) - 2):
                        
                        #El tiempo que resto a la linerna siempre será de la persona que tarda más
                        tiempo_en_cruzar =  max(self.tiempo_personas[i], self.tiempo_personas[j])
                        
                        #Hago una comprobación extra para evitar caer en un estado en el que se me ha gastado el tiempo de la linterna
                        if((state[j] == self.derecha) and (tiempo_en_cruzar <= state[len(state) - 1])):
                            accs.append((i, j))
    
        #Si la linterna se encuentra a la izquierda del puente
        else:
            #Un único bucle para recorrer a las personas
            for i in range(len(state) - 2):
                
                tiempo_en_cruzar = self.tiempo_personas[i]
                
                #Siempre y cuando el tiempo de la linterna sea mayor o igual que el tiempo que me cuesta
                if((state[i] == self.izquierda) and (tiempo_en_cruzar <= state[len(state) - 1])):
                    accs.append((i, i))
                    
        return accs
                        
    def result(self, state, accion):

        l = list(state)
        
        #Extraigo los índices de las personas que van a cruzar
        indice1 = accion[0]
        indice2 = accion[1]
        
        #Si la linterna está a la derecha, siempre pasan 2 personas
        if(l[len(l) - 2] == self.derecha):
            
            #El tiempo máximo de los dos se lo resto a la lintena
            l[len(l) - 1] -= max(self.tiempo_personas[indice1], self.tiempo_personas[indice2])
            
            #Actualizo ubicaciones de las personas y de la linerna
            l[indice1] = 1
            l[indice2] = 1
            l[len(l) - 2] = 1
            
        else:
            
            #Le resto el tiempo que tarda a la linterna
            l[len(l) - 1] -= self.tiempo_personas[indice1]
            
            #Actualizo ubicaciones de las personas y de la linerna
            l[indice1] = 0
            l[len(l) - 2] = 0
        
        return tuple(l)
        
    
    def goal_test(self, state):
        
        #Compruebo que todos estén a la izquierda, es decir, que si hay alguno a la derecha no es solución
        for i in range(len(state) - 1):
            if(state[i] == self.derecha):
                return False
            
        return True    

# PROBLEMA 1: EJEMPLO DE BÚSQUEDA

__EJEMPLO DE LA HOJA__

In [19]:
estado = Linterna((10, 30, 60, 80, 120, 300))

In [20]:
estado.initial

(0, 0, 0, 0, 0, 0, 300)

In [21]:
estado.actions(estado.initial)

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4)]

In [22]:
breadth_first_graph_search(estado).solution()

[(0, 1), (0, 0), (0, 2), (0, 0), (3, 4), (1, 1), (0, 1)]

__OTRO EJEMPLO CON MÁS PERSONAS__

In [23]:
estado = Linterna((10, 30, 60, 80, 120, 240, 490, 3000))

In [24]:
estado.actions(estado.initial)

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (3, 4),
 (3, 5),
 (3, 6),
 (4, 5),
 (4, 6),
 (5, 6)]

In [25]:
depth_first_graph_search(estado).solution()

[(5, 6),
 (6, 6),
 (4, 6),
 (6, 6),
 (3, 6),
 (4, 4),
 (2, 4),
 (4, 4),
 (1, 4),
 (1, 1),
 (0, 1)]

# PROBLEMA 2: PUZZLE INFANTIL

__Introducción y entrada del problema:__

Este problema consiste en una serie de operaciones en la que hay que rellenar unos huecos con números para obtener el resultado deseado. En el ejemplo, tenemos que rellenar 9 huecos con números que no se repitan del 1 al 9, pero esta implementación podría ser aplicable a problemas con más o menos huecos y con diferentes operaciones. Si quisieramos tener n huecos, las posibilidades de números irían desde 1 a n. En nuestro caso, y asumiendo que nunca se van a poder dar ceros como números validos, nuestra entrada del problema es una tupla en la que los huecos se representan con 0's, los números como su propio valor, y las operaciones como caracteres. Es importante que la entrada esté en el orden índicado en el que queremos que se resuelva. Además, también deberá incluir la solución. Por ejemplo, en el caso de la hoja sería: PuzzleEcuacion((0,'+',13,'x',0,':',0,'+',0,'+',12,'x',0,'-',0,'-',11,'+',0,'x',0,':',0,'-',10, '=', 66)).

__Representación de los estados:__

Los estados están representados como una tupla de tamaño equivalente al número de huecos contenido en nuestro porblema. Inicialmente esta tupla estará llena de ceros, y a medida que se van generando hijos, se va sustituyendo el cero más a la izquierda por un número. Es decir, que en cada hijo vamos 'colocando' un dígito diferente a los que ya han aparecido. Además de esto, contamos con un array que contiene todo el problema completo y el resultado, ambos estáticos ya que no se modifican.

__Función init:__

Dada una entrada inicial con el puzzle completo, extraemos el resultado y creamos dos arrays. Uno de ellos va a contener todo el puzzle excepto el resultado y el otro los huecos que hay en él. Para ello recorremos la entrada y si encuentro un 0, lo añado al array de los huecos y en todos los casos añado al array inicial el elemento que toca (Ya sea hueco, número u operador). Al final, simplemente cambio el array de huecos por una tupla que representará el estado.

__Función actions:__

En esta función, analizaremos el estado actual y devolveremos los valores posibles de 1 a n que podremos colocar en la posición i, es decir, donde esté el primer hueco por la izquierda. Para ello se va recorriendo el estado hasta encontrar el primer hueco y mientras tanto, voy marcando los valores (en un array de booleanos) que aparecen para que las opciones sean siempre valores distintos. Después de tener completo el array de marcas, lo recorro para ver que números no han aparecido y añadirlos a las opciones posibles. Cabe destacar que las posiciones del array de marcas corresponden a los valores posibles - 1, es decir, que si los valores posibles van de 1 a 9, los índices de este array irán de 0 a 8. En definitiva, si un número ha aparecido tendrá valor True y False en caso contrario.

__Función result:__

Esta función inserta el dígito contenido en action al hueco más próximo por la izquierda. Simplemente tiene un bucle que recorre el estado y encuentra la posición donde debe insertarlo. Como mejora añadida, hace uso del array de marcas y en el caso en el que haya insertado el penúltimo elemento, es trivial que sólo tengo un único elemento disponible para insertar en la última posición. Dicho esto, hace uso de un array de marcas que se rellena mientras se recorre para saber cual es el último elemento que falta por colocar.


__Función goal_test:__

El objetivo principal de esta función es comprobar si nos dá el resultado obtenido. Para ello realizo una fusión entre el array_inicial y el estado actual. Es decir, completo los huecos con los valores que he ido colocando. En un primer bucle, voy insertando en un array auxiliar el puzzle completo sustituyendo los huecos por los valores de mi estado. Posteriormente realizo un recorrido sobre el array resultante para comprobar si hay huecos, en ese caso la solución ya no es válida y retorno False. Una vez pasado ese filtro, lo único que hago es recorrer el array para hallar la solución que obtengo con los valores de mi estado. En cada iteración, opero la variable res utilizando un operador junto al valor siguiente al mismo. Al final, si coinciden los resultados, puedo decir que es solución.

In [26]:
class PuzzleEcuacion(Problem):
    
    #Array con el puzzle completo que jamás será modificado salvo en la inicialización del problema
    array_inicial = []
    
    #Variable donde vamos a guardar todas las variables
    resultado = 0
    
    def __init__(self, entrada_inicial):
        
        #Extraigo el resultado de la entrada inicial
        self.resultado = entrada_inicial[len(entrada_inicial) - 1]
        
        #Array estático inicial
        self.array_inicial = []
        
        #Array donde voy a guardar los estados
        array_huecos = []
        
        #Bucle que va a recorrer toda la entrada inicial
        for i in range(len(entrada_inicial) - 2):
            
            #Si es un hueco (vale 0), lo añado a la tupla del estado
            if(entrada_inicial[i] == 0):
                array_huecos.append(0)
            
            #En todos los casos guardo el puzzle completo en array_inicial
            self.array_inicial.append(entrada_inicial[i])
        
        #Paso a una tupla el array con los huecos
        huecos = tuple(array_huecos)
        self.initial = huecos
        
    
    def actions(self, state):
        
        accs = list()
        
        #Array en el que iremos marcando los números que aparecen
        #Las posiciones del array irán de 0 a n-1, sin embargo los números van de 1 a n
        marcados = []
        
        #Inicializo a false en función del número de huecos
        for i in range(len(state)):
            marcados.append(False)
        
        #Valor booleano que indica cuando he llegado a un hueco, es decir, a un 0
        encontrado = False
        
        #Índice utilizado para recorrer el bucle y utilizarlos como posición para el array de marcas
        j = 0
        
        #Bucle en el que cuando encuentre un 0, termino la búsqueda
        while(encontrado == False and j < len(state)):
            
            #Si el elemento no es hueco, marco la posición correspondiente al valor
            if(state[j] > 0):
                marcados[state[j] - 1] = True
            
            #En otro caso, estoy en un hueco
            else:
                encontrado = True
                
            j += 1
        
        #Bucle que irá añadiendo a las opciones posibles los números que no han aparecido
        for i in range(len(marcados)):
            
            if(marcados[i] == False):
                accs.append(i + 1)
            
        return accs
       
        
    def result(self, state, action):
        
        l = list(state)
        
        #Array de marcas utilizado únicamente en el caso en el que sólo me quede un hueco por poner
        #Simplemente es una mejora para evitar crear más estados, ya que si solo queda un hueco, sabemos que valor irá en él
        marcados = []
        
        #Inicialización array marcas
        for i in range(len(state)):
            marcados.append(False)
        
        #Booleano que indica dónde está el hueco
        encontrado = False
        
        #Índice para ver dónde debo insertar el siguiente dígito
        i = 0
        
        #Bucle principal que recorre el estado actual e inserta el valor correspondiente a la accion en el siguiente hueco
        while(encontrado == False and i < len(state)):
            if(l[i] == 0):
                encontrado = True
                l[i] = action
            else:
                marcados[i-1] = True
            i += 1
        
        #Sólo entra en el caso en el que sólo quede un número por poner
        if(i == (len(state) - 1)):
            
            #Índice array de marcas
            j = 0
            
            #Hace uso del array de marcas para saber cual es
            while(marcados[j] == True):
                j += 1
            
            #Inserto el índice del array de marcas + 1
            l[i] = j+1
            
        return tuple(l)

    
    def goal_test(self, state):
        
        #Array local en la que voy a meter el puzzle completo con los huecos rellenos
        array_total = []
        
        #Índice para recorrer el estado
        j = 0
        
        #Bucle que recorre el array inicial del puzzle
        for i in range(len(self.array_inicial)):
            
            #Si es 0, inserto al array_total el valor correspondiente del estado
            if(self.array_inicial[i] == 0):
                array_total.append(state[j])
                j += 1
            
            #En otro caso, no es un hueco y lo inserto en array_total
            else:
                array_total.append(self.array_inicial[i])
        
        #Si aparece algún 0 en el array, significa que aún tengo huecos que rellenar, por lo que devuelvo False
        for i in range(len(self.array_inicial)):
            if(array_total[i] == 0):
                return False
        
        #Variable resultado, inicializada con el primer número que aparece
        res = array_total[0]
        
        #Índice para recorrer el array_total
        j = 1
        
        #Bucle que calcula la solución, para ello opera sobre la variable res en función del siguiente operador
        while(j < len(array_total) - 1):
            
            if(array_total[j] == '+'):
                res += array_total[j + 1]
              
            elif(array_total[j] == '-'):
                res -= array_total[j + 1]
              
            elif(array_total[j] == 'x'):
                res *= array_total[j + 1]
              
            if(array_total[j] == ':'):
                res /= array_total[j + 1]
            
            #En cada iteración, utilizo un operador y un valor
            j += 2
        
        #Si el resultado es el esperado, devuelvo True
        return self.resultado == res

# PROBLEMA 2: EJEMPLO DE BÚSQUEDA

__EJEMPLO DE LA HOJA__

In [27]:
ec = PuzzleEcuacion((0,'+',13,'x',0,':',0,'+',0,'+',12,'x',0,'-',0,'-',11,'+',0,'x',0,':',0,'-',10, '=', 66))

In [28]:
depth_first_graph_search(ec).solution()

[9, 8, 6, 7, 3, 2, 1, 4]